In [1]:
import numpy as np, scipy, matplotlib.pyplot as plt, IPython.display as ipd, sklearn
import librosa, librosa.display
import os
import warnings
import tqdm
import json
import pickle
import pandas as pd

warnings.filterwarnings('ignore')

In [2]:
SOUND_FOLDER_training = "./Audio_Dataset/training_data/"
SOUND_FOLDER_testing = "./Audio_Dataset/testing_data/"
FEAT_FOLDER = "./Features"

In [3]:
# Utilities

def sigmoid_function(x):
    return 1.0/(1.0+np.exp(-x))


---
# **FEATURE EXTRACTION**




In [4]:
# Feature Extraction

def extract_mfcc(y, sr):
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=22)
    mfcc_mean = np.mean(mfcc, axis=1)
    return mfcc, mfcc_mean


In [5]:
path_training = os.path.join(SOUND_FOLDER_training)
path_testing = os.path.join(SOUND_FOLDER_testing)

In [6]:
# Retrieve list of files
list_of_files_training = []
list_of_files_testing = []

# dirs=directories
for (root, dirs, file) in os.walk(path_training):
    for f in file:
        if '.wav' in f:
            list_of_files_training.append(root+"/"+f)

for (root, dirs, file) in os.walk(path_testing):
    for f in file:
        if '.wav' in f:
            list_of_files_testing.append(root+"/"+f)

print(list_of_files_training)
print(len(list_of_files_training)) # 100 files = 50 disco + 50 hiphop
print(list_of_files_testing)
print(len(list_of_files_testing))

['./Audio_Dataset/training_data/metal/metal.00022.wav', './Audio_Dataset/training_data/metal/metal.00036.wav', './Audio_Dataset/training_data/metal/metal.00037.wav', './Audio_Dataset/training_data/metal/metal.00023.wav', './Audio_Dataset/training_data/metal/metal.00009.wav', './Audio_Dataset/training_data/metal/metal.00035.wav', './Audio_Dataset/training_data/metal/metal.00021.wav', './Audio_Dataset/training_data/metal/metal.00020.wav', './Audio_Dataset/training_data/metal/metal.00034.wav', './Audio_Dataset/training_data/metal/metal.00008.wav', './Audio_Dataset/training_data/metal/metal.00030.wav', './Audio_Dataset/training_data/metal/metal.00024.wav', './Audio_Dataset/training_data/metal/metal.00018.wav', './Audio_Dataset/training_data/metal/metal.00019.wav', './Audio_Dataset/training_data/metal/metal.00025.wav', './Audio_Dataset/training_data/metal/metal.00031.wav', './Audio_Dataset/training_data/metal/metal.00027.wav', './Audio_Dataset/training_data/metal/metal.00033.wav', './Audio_

In [7]:
# Define the Dataframes of features
feat_df_training = pd.DataFrame(columns=['file_name'])
feat_df_testing = pd.DataFrame(columns=['file_name'])

In [8]:
# -- FEATURE EXTRACTION --

# for TRAINING data

for fname in tqdm.tqdm(list_of_files_training):
  signal, sr = librosa.load(path=fname)

  db_new_row = {'path': fname}
  db_new_row['file_name'] = os.path.basename(fname).split('/')[-1]
  
  # MFCC
  feat, feat_mean = extract_mfcc(y=signal, sr=sr)
  for i, val in enumerate(feat_mean):
    db_new_row["MFCC_mean_"+str(i)] = val

  s = pd.Series(db_new_row)
  feat_df_training = pd.concat([feat_df_training, s.to_frame().T], ignore_index=True)

  0%|          | 0/160 [00:00<?, ?it/s]

100%|██████████| 160/160 [00:06<00:00, 23.12it/s]


In [9]:
# -- FEATURE EXTRACTION --

# for TESTING data

for fname in tqdm.tqdm(list_of_files_testing):
  signal, sr = librosa.load(path=fname)

  db_new_row = {'path': fname}
  db_new_row['file_name'] = os.path.basename(fname).split('/')[-1]
  
  # MFCC
  feat, feat_mean = extract_mfcc(y=signal, sr=sr)
  for i, val in enumerate(feat_mean):
    db_new_row["MFCC_mean_"+str(i)] = val

  s = pd.Series(db_new_row)
  feat_df_testing = pd.concat([feat_df_testing, s.to_frame().T], ignore_index=True)

100%|██████████| 40/40 [00:01<00:00, 26.13it/s]


In [10]:
feat_df_training

,file_name,path,MFCC_mean_0,MFCC_mean_1,MFCC_mean_2,MFCC_mean_3,MFCC_mean_4,MFCC_mean_5,MFCC_mean_6,MFCC_mean_7,...,MFCC_mean_12,MFCC_mean_13,MFCC_mean_14,MFCC_mean_15,MFCC_mean_16,MFCC_mean_17,MFCC_mean_18,MFCC_mean_19,MFCC_mean_20,MFCC_mean_21
0,metal.00022.wav,./Audio_Dataset/training_data/metal/metal.0002...,-45.855606,107.999084,-11.737645,66.326881,-30.206829,9.242658,-25.778023,21.871576,...,-14.23165,-2.229446,-5.779099,-5.861246,-2.548104,1.480481,-5.618702,-2.821037,-7.000286,0.963876
1,metal.00036.wav,./Audio_Dataset/training_data/metal/metal.0003...,-71.290672,82.339409,-30.162804,56.456169,-10.86737,26.810652,-15.801576,25.331217,...,-14.269172,9.53343,-10.371092,8.087852,-7.460728,5.422095,-4.396084,3.044864,-6.157757,0.448798
2,metal.00037.wav,./Audio_Dataset/training_data/metal/metal.0003...,-79.104179,106.81736,-38.745987,39.853371,-26.754459,32.471489,-15.025887,29.701759,...,-3.071116,3.566604,-17.278639,-2.124718,-12.567954,-1.238482,-1.207811,9.03112,-5.958776,1.53266
3,metal.00023.wav,./Audio_Dataset/training_data/metal/metal.0002...,-43.797508,82.774704,-29.60919,70.397179,-21.588844,31.144138,2.090371,28.873167,...,-7.368736,7.88214,-13.857293,4.057312,-6.865944,-1.79484,1.459278,4.533475,-7.945553,-1.482065
4,metal.00009.wav,./Audio_Dataset/training_data/metal/metal.0000...,3.638441,104.756203,-0.833012,55.920498,-29.22942,31.247383,-14.18906,20.214329,...,-12.991373,15.278262,-6.923676,9.862301,-6.496095,4.899212,-0.778939,6.45774,-5.718965,2.648783
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,classical.00075.wav,./Audio_Dataset/training_data/classical/classi...,-413.00116,138.531708,-19.246777,34.156403,8.985244,-4.293563,4.150046,0.869279,...,5.919068,3.498136,6.624698,6.735387,2.467337,-1.875985,-0.05678,2.725177,-1.318019,-4.69021
156,classical.00049.wav,./Audio_Dataset/training_data/classical/classi...,-98.353577,98.355499,-20.490532,18.820034,-2.095781,2.665037,-7.311517,-3.590114,...,-2.593085,-1.470157,-3.169344,0.658032,-4.818465,-2.221544,-3.810496,0.234092,-3.702766,-5.138118
157,classical.00048.wav,./Audio_Dataset/training_data/classical/classi...,-124.045258,148.595657,-21.581469,17.421335,-1.415515,4.123947,-7.020776,-5.144728,...,-0.900143,-2.414657,-4.325914,-0.872547,-4.684666,-2.250636,-5.178997,-3.217605,-6.263325,-2.741918
158,classical.00074.wav,./Audio_Dataset/training_data/classical/classi...,-289.741272,143.902832,-27.002756,24.065563,-1.146928,-6.300432,-2.745879,-3.524174,...,3.075898,2.683342,2.52569,4.61217,2.380265,-1.272908,-3.553611,-2.813544,3.212688,6.503239


In [11]:
feat_df_testing

,file_name,path,MFCC_mean_0,MFCC_mean_1,MFCC_mean_2,MFCC_mean_3,MFCC_mean_4,MFCC_mean_5,MFCC_mean_6,MFCC_mean_7,...,MFCC_mean_12,MFCC_mean_13,MFCC_mean_14,MFCC_mean_15,MFCC_mean_16,MFCC_mean_17,MFCC_mean_18,MFCC_mean_19,MFCC_mean_20,MFCC_mean_21
0,metal.00082.wav,./Audio_Dataset/testing_data/metal/metal.00082...,-57.683716,101.418671,-41.490356,55.114075,-23.339161,28.144377,-12.126061,18.141899,...,-12.276187,4.799379,-12.402426,5.41698,-8.848057,0.046606,-3.173728,-3.367191,-7.77986,-2.798162
1,metal.00096.wav,./Audio_Dataset/testing_data/metal/metal.00096...,-51.620953,86.171623,-20.278294,59.063999,-10.875287,27.535135,-11.259489,23.944517,...,-10.735527,8.389958,-7.104511,8.215934,-6.18395,3.794329,-1.912575,0.892365,-5.941866,-1.267345
2,metal.00097.wav,./Audio_Dataset/testing_data/metal/metal.00097...,-82.130188,98.281494,-16.521841,68.434624,1.434923,29.857725,-5.28196,24.396568,...,-8.499245,6.674568,-6.288647,4.442029,-7.257452,2.318223,4.238797,1.099661,-5.512079,-3.627356
3,metal.00083.wav,./Audio_Dataset/testing_data/metal/metal.00083...,-29.381208,101.875473,-29.112329,69.73011,-16.168983,19.438528,-18.33918,19.062609,...,-14.794669,4.604305,-12.242908,5.262515,-10.682246,6.413686,-6.407439,-7.017361,-3.823075,-2.85165
4,metal.00095.wav,./Audio_Dataset/testing_data/metal/metal.00095...,-35.46846,94.100639,-24.217218,43.315338,-16.380928,29.513052,-5.864533,28.445436,...,-8.49749,11.400663,-6.135518,9.322351,-6.610096,3.50287,-3.287504,3.65741,-2.049653,1.562598
5,metal.00081.wav,./Audio_Dataset/testing_data/metal/metal.00081...,-19.165882,118.840714,-52.14489,42.700321,-21.572849,27.6777,-14.611529,27.019527,...,-4.661488,1.085177,-5.038619,4.051173,-6.416048,1.992068,-5.504744,0.453913,-6.249013,-1.264264
6,metal.00080.wav,./Audio_Dataset/testing_data/metal/metal.00080...,-73.529945,74.658463,-16.444176,55.64547,-7.73465,36.538662,-18.304913,27.820959,...,-7.292049,9.596542,-6.747253,7.88831,-3.577514,2.662014,-5.100033,3.033263,-1.774702,1.828035
7,metal.00094.wav,./Audio_Dataset/testing_data/metal/metal.00094...,-104.723763,87.753723,-33.248859,48.474255,-28.912521,22.51424,-24.980129,20.53738,...,-13.891189,10.281917,-4.697016,9.941506,-5.62195,4.79159,-11.673745,2.961579,-0.693878,0.864571
8,metal.00090.wav,./Audio_Dataset/testing_data/metal/metal.00090...,-93.544128,100.646156,-23.339111,52.490635,-7.746713,39.590942,-7.766726,26.725874,...,-9.955353,4.162056,-12.794265,1.787793,-2.017398,1.936224,-9.054855,-7.048695,-11.294311,-3.900863
9,metal.00084.wav,./Audio_Dataset/testing_data/metal/metal.00084...,0.25672,94.185646,-32.401512,45.452942,-15.199275,25.886078,-12.390217,19.618542,...,-8.688227,6.829992,-8.352292,4.972199,-4.337806,4.781101,-4.180623,2.548584,-1.330388,2.565098


---
# **SAVE FEATURE FILES**


In [12]:
path = os.path.join(FEAT_FOLDER)
print("save features in " + path)

isExist = os.path.exists(path)
if not isExist:

   # Create a new directory because it does not exist
   os.makedirs(path)
   print("The new directory is created!")

print("save features to -> "+path + "feature_training.pkl")
with open(os.path.join(path, "feature_training.pkl"), 'wb') as fout:
   pickle.dump(feat_df_training, fout, protocol=pickle.HIGHEST_PROTOCOL)

print("save features to -> "+path + "feature_testing.pkl")
with open(os.path.join(path, "feature_testing.pkl"), 'wb') as fout:
   pickle.dump(feat_df_testing, fout, protocol=pickle.HIGHEST_PROTOCOL)

save features in ./Features
save features to -> ./Featuresfeature_training.pkl
save features to -> ./Featuresfeature_testing.pkl
